## 13.3 Offline Caching

W tej aplikacji zobaczymy w jaki sposób wykonać *offline caching* z wykorzystaniem lokalnej bazy danych. Zadanie polega na wykonaniu *snapshotu* danych dostępnych z zewnętrznego serwisu. Idea *offline cachingu* zakłada że urządzenie docelowe może nie mieć stałego dostępu do sieci. W przypadku braku dostępu, wykorzystujemy wersję danych dostępnych w lokalnej bazie.

W aplikacji wykorzystamy https://random-data-api.com/, który przy każdym żądaniu generuje losowe dane, wykorzystamy *endpoint* `\users` generując każdorazowo dane 20 użytkowników. W tej wersji chcemy całkowicie zastąpić istniejące dane w lokalnej bazie ich nową wersją.

Podejście to spełnia warunki *single source of truth* - pojedynczego źródła danych dostarczanych użytkownikowi. Będziemy wyświetlać tylko dane pochodzące z lokalnej bazy.

W aplikacji wykorzystamy architekturę **MVVM** wraz z bibliotekami `Dagger-Hilt`, `Retrofit`, `ROOM`, oraz `Glide`. Będzie zawierać pojedynczą aktywność na której umieścimy `RecyclerView` - tutaj nie będziemy wykorzystywać komponentu `Jetpack Navigation`.

<img src="https://media1.giphy.com/media/y0RLxRN7SPJ7mI7Hw5/giphy.gif?cid=790b7611ea236e1ec92b9220e3629a80e851a7759882cbee&rid=giphy.gif&ct=g" width="200" />

Rozpocznijmy od przygotowania modelu danych, odpowiedź serwera wygląda następująco.

In [ ]:
{
   "id":4634,
   "uid":"e1c88b6e-800c-44fe-b45a-317d6311ea32",
   "password":"rKbRvTZGIL",
   "first_name":"Scott",
   "last_name":"Veum",
   "username":"scott.veum",
   "email":"scott.veum@email.com",
   "avatar":"https://robohash.org/quianihilexercitationem.png?size=300x300\u0026set=set1",
   "gender":"Non-binary",
   "phone_number":"+420 573-352-4638 x18831",
   "social_insurance_number":"968138909",
   "date_of_birth":"1993-03-23",
   "employment":{
      "title":"Senior Agent",
      "key_skill":"Networking skills"
   },
   "address":{
      "city":"Juniorshire",
      "street_name":"Pacocha Dale",
      "street_address":"51943 Earnest Parks",
      "zip_code":"26716-5002",
      "state":"Mississippi",
      "country":"United States",
      "coordinates":{
         "lat":-56.16662202146844,
         "lng":68.36895424808517
      }
   },
   "credit_card":{
      "cc_number":"4135-2456-4633-3923"
   },
   "subscription":{
      "plan":"Premium",
      "status":"Idle",
      "payment_method":"Credit card",
      "term":"Payment in advance"
   }
}

Na jej podstawie możemy zamodelować dane - można wykorzystać dowolny dostępny plugin, czy stronę internetową.

In [ ]:
public class User {
    private int id;
    private String uid;
    private String password;
    private String first_name;
    private String last_name;
    private String username;
    private String email;
    private String avatar;
    private String gender;
    private String phone_number;
    private String social_insurance_number;
    private String date_of_birth;
    private Employment employment;
    private Address address;
    private CreditCard credit_card;
    private Subscription subscription;

    public User(int id,
                String uid,
                String password,
                String first_name,
                String last_name,
                String username,
                String email,
                String avatar,
                String gender,
                String phone_number,
                String social_insurance_number,
                String date_of_birth,
                Employment employment,
                Address address,
                CreditCard credit_card,
                Subscription subscription) {
        this.id = id;
        this.uid = uid;
        this.password = password;
        this.first_name = first_name;
        this.last_name = last_name;
        this.username = username;
        this.email = email;
        this.avatar = avatar;
        this.gender = gender;
        this.phone_number = phone_number;
        this.social_insurance_number = social_insurance_number;
        this.date_of_birth = date_of_birth;
        this.employment = employment;
        this.address = address;
        this.credit_card = credit_card;
        this.subscription = subscription;
    }

    public int getId() {
        return id;
    }

    public void setId(int id) {
        this.id = id;
    }

    public String getUid() {
        return uid;
    }

    public void setUid(String uid) {
        this.uid = uid;
    }

    public String getPassword() {
        return password;
    }

    public void setPassword(String password) {
        this.password = password;
    }

    public String getFirst_name() {
        return first_name;
    }

    public void setFirst_name(String first_name) {
        this.first_name = first_name;
    }

    public String getLast_name() {
        return last_name;
    }

    public void setLast_name(String last_name) {
        this.last_name = last_name;
    }

    public String getUsername() {
        return username;
    }

    public void setUsername(String username) {
        this.username = username;
    }

    public String getEmail() {
        return email;
    }

    public void setEmail(String email) {
        this.email = email;
    }

    public String getAvatar() {
        return avatar;
    }

    public void setAvatar(String avatar) {
        this.avatar = avatar;
    }

    public String getGender() {
        return gender;
    }

    public void setGender(String gender) {
        this.gender = gender;
    }

    public String getPhone_number() {
        return phone_number;
    }

    public void setPhone_number(String phone_number) {
        this.phone_number = phone_number;
    }

    public String getSocial_insurance_number() {
        return social_insurance_number;
    }

    public void setSocial_insurance_number(String social_insurance_number) {
        this.social_insurance_number = social_insurance_number;
    }

    public String getDate_of_birth() {
        return date_of_birth;
    }

    public void setDate_of_birth(String date_of_birth) {
        this.date_of_birth = date_of_birth;
    }

    public Employment getEmployment() {
        return employment;
    }

    public void setEmployment(Employment employment) {
        this.employment = employment;
    }

    public Address getAddress() {
        return address;
    }

    public void setAddress(Address address) {
        this.address = address;
    }

    public CreditCard getCredit_card() {
        return credit_card;
    }

    public void setCredit_card(CreditCard credit_card) {
        this.credit_card = credit_card;
    }

    public Subscription getSubscription() {
        return subscription;
    }

    public void setSubscription(Subscription subscription) {
        this.subscription = subscription;
    }
}

public class Address {
    private String city;
    private String street_name;
    private String street_address;
    private String zip_code;
    private String state;
    private String country;

    public Address(String city,
                   String street_name,
                   String street_address,
                   String zip_code,
                   String state,
                   String country) {
        this.city = city;
        this.street_name = street_name;
        this.street_address = street_address;
        this.zip_code = zip_code;
        this.state = state;
        this.country = country;
    }

    public String getCity() {
        return city;
    }

    public void setCity(String city) {
        this.city = city;
    }

    public String getStreet_name() {
        return street_name;
    }

    public void setStreet_name(String street_name) {
        this.street_name = street_name;
    }

    public String getStreet_address() {
        return street_address;
    }

    public void setStreet_address(String street_address) {
        this.street_address = street_address;
    }

    public String getZip_code() {
        return zip_code;
    }

    public void setZip_code(String zip_code) {
        this.zip_code = zip_code;
    }

    public String getState() {
        return state;
    }

    public void setState(String state) {
        this.state = state;
    }

    public String getCountry() {
        return country;
    }

    public void setCountry(String country) {
        this.country = country;
    }
}

public class CreditCard {
    private String cc_number;

    public CreditCard(String cc_number) {
        this.cc_number = cc_number;
    }

    public String getCc_number() {
        return cc_number;
    }

    public void setCc_number(String cc_number) {
        this.cc_number = cc_number;
    }
}

public class Employment {
    private String title;
    private String key_skill;

    public Employment(String title) {
        this.title = title;
    }

    public String getTitle() {
        return title;
    }

    public void setTitle(String title) {
        this.title = title;
    }
}


public class Subscription {
    private String plan;
    private String status;
    private String payment_method;
    private String term;

    public Subscription(String plan, String status, String payment_method, String term) {
        this.plan = plan;
        this.status = status;
        this.payment_method = payment_method;
        this.term = term;
    }

    public String getPlan() {
        return plan;
    }

    public void setPlan(String plan) {
        this.plan = plan;
    }

    public String getStatus() {
        return status;
    }

    public void setStatus(String status) {
        this.status = status;
    }

    public String getPayment_method() {
        return payment_method;
    }

    public void setPayment_method(String payment_method) {
        this.payment_method = payment_method;
    }

    public String getTerm() {
        return term;
    }

    public void setTerm(String term) {
        this.term = term;
    }
}


Posiadając model danych, możemy przygotować layout elementu `RecyclerView` oraz głównej aktywności.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".ui.MainActivity">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/recyclerView"
        android:layout_margin="8dp"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <ProgressBar
        android:id="@+id/progressBar"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:layout_marginTop="36dp"
        android:background="@android:color/transparent"
        android:visibility="invisible"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_margin="8dp"
    android:orientation="vertical">

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

        <TextView
            android:id="@+id/firstName"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="1"
            android:text="TextView"
            android:textSize="24sp"
            app:layout_constraintStart_toStartOf="parent"
            tools:layout_editor_absoluteY="22dp" />

        <TextView
            android:id="@+id/lastName"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="1"
            android:text="TextView"
            android:textSize="24sp"
            app:layout_constraintEnd_toEndOf="parent"
            tools:layout_editor_absoluteY="20dp" />
    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

        <LinearLayout
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:orientation="vertical">

            <TextView
                android:id="@+id/username"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:text="TextView" />

            <TextView
                android:id="@+id/password"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:text="TextView" />

            <TextView
                android:id="@+id/email"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:text="TextView" />

            <TextView
                android:id="@+id/gender"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:text="TextView" />

            <TextView
                android:id="@+id/phone"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:text="TextView" />

            <TextView
                android:id="@+id/dateOfBirth"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:text="TextView" />

            <TextView
                android:id="@+id/employment"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:text="TextView" />


        </LinearLayout>

        <ImageView
            android:id="@+id/image"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:src="@drawable/ic_android_black_24dp"
            android:layout_height="wrap_content"
            android:contentDescription="user image" />
    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="8dp"
        android:orientation="horizontal">

        <TextView
            android:id="@+id/country"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />

        <TextView
            android:id="@+id/city"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />

        <TextView
            android:id="@+id/state"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />
    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="8dp"
        android:orientation="horizontal">

        <TextView
            android:id="@+id/street_name"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />

        <TextView
            android:id="@+id/street_address"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />

        <TextView
            android:id="@+id/zip_code"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />
    </LinearLayout>

    <TextView
        android:id="@+id/creditCardNumber"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="TextView" />

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="8dp"
        android:orientation="horizontal">

        <TextView
            android:id="@+id/plan"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />

        <TextView
            android:id="@+id/status"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />

        <TextView
            android:id="@+id/payment_method"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />

        <TextView
            android:id="@+id/term"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />
    </LinearLayout>
</LinearLayout>

Dodajmy interfejs `RandomApi` z jedną metodą, która zwraca listę 20 użytkowników

In [ ]:
public interface RandomApi {
    @GET("users?size=20")
    Call<List<User>> users();
}

Dodajmy `AppModule` z metodą dostarczającą instancję obiektu o typie `RandomApi`.

In [ ]:
@Module
@InstallIn(SingletonComponent.class)
public class AppModule {

    @Provides
    @Singleton
    RandomApi provideRandomApi(){
        HttpLoggingInterceptor interceptor = new HttpLoggingInterceptor();
        interceptor.setLevel(HttpLoggingInterceptor.Level.BODY);
        OkHttpClient client = new OkHttpClient.Builder()
                .addInterceptor(interceptor)
                .build();
        return new Retrofit.Builder()
                .baseUrl("https://random-data-api.com/api/v2/")
                .addConverterFactory(GsonConverterFactory.create())
                .client(client)
                .build().create(RandomApi.class);
    }
}

Przejdźmy do utworzenia lokalnej bazy danych - tutaj od razu napotykamy problem. Bazy danych mogą przyjmować tylko określone typy, nie mogą przechowywać obiektów niestandardowych. Klasa `User` posiada obiekty o typach `Address`, `CreditCard`, `Employment` i `Subsciption`. Aby zapisać instancje tych obiektów w bazie `ROOM`, musimy je przekonwertować do typu standardowego - tutaj wyborem jest `String`.

Zaimplementujemy kilka klas - dla każdego typu - opisujących sposób konwersji. Rozpocznijmy od `AddressConverter`.

In [ ]:
public class AddressConverter {

Klasa będzie posiadała dwie metody - dla konwersji `Address` -> `String`, oraz dla konwersji `String` -> `Address`.

Samą konwersję można przeprowadzić na szereg różnych sposobów - tutaj wykorzystamy format `JSON`

In [ ]:
@TypeConverter
public static String fromAddress(Address address){
    Gson gson = new Gson();
    return gson.toJson(address);
}

Stosujemy adnotacje
- `@TypeConverter` - adnotacja oznacza metody wykorzystywane do konwersji - samą konwersję `ROOM` przeprowadza automatycznie

Potrzebujemy jeszcze drugą metodę konwersji.

In [ ]:
@TypeConverter
public static Address toAddress(String address){
    Gson gson = new Gson();
    return gson.fromJson(address, Address.class);
}

Dodajmy pozostałe klasy konwerterów.

In [ ]:
public class CreditCardConverter {

    @TypeConverter
    public static String fromCreditCard(CreditCard creditCard){
        return creditCard.getCc_number();
    }

    @TypeConverter
    public static CreditCard toCreditCard(String creditCard){
        return new CreditCard(creditCard);
    }
}

public class EmploymentConverter {
    @TypeConverter
    public static String fromEmployment(Employment employment){
        return employment.getTitle();
    }

    @TypeConverter
    public static Employment toEmployment(String employment){
        return new Employment(employment);
    }
}

public class SubscriptionConverter {
    @TypeConverter
    public static String fromSubscription(Subscription subscription){
        Gson gson = new Gson();
        return gson.toJson(subscription);
    }

    @TypeConverter
    public  static Subscription toSubscription(String subscription){
        Gson gson = new Gson();
        return gson.fromJson(subscription, Subscription.class);
    }
}

Przejdźmy do implementacji samej bazy, oznaczmy klasę `User` jako `@Entity` oraz `id` jako `@PrimaryKey`.

In [ ]:
@Entity(tableName = "users")
public class User {
    @PrimaryKey
    private int id;
    ...
}

Zdefiniujmy `Dao` z trzema metodami

In [ ]:
@Dao
public interface UserDao {
    @Insert(onConflict = REPLACE)
    void insert(List<User> users);

    @Query("DELETE FROM users")
    void clear();

    @Query("SELECT * FROM users")
    LiveData<List<User>> getUsers();
}

Ponieważ chcemy zastępować całą listę w bazie, potrzebujemy dwóch metod
- `clear` - czyści bazę
- `instert` - dodaje nową listę

Mamy również metodę zwracającą całą listę jako `LiveData`.

Dodajmy klasę abstrakcyjną `UserDatabase`, wykorzystamy adnotację `TypeConverters` do zakomunikowania `ROOM` że wymagane jest wykorzystanie konwerterów, oraz podamy jawnie wszystkie obiekty konwerterów.

In [ ]:
@Database(entities = {User.class}, version = 1, exportSchema = false)
@TypeConverters({
    AddressConverter.class, 
    CreditCardConverter.class, 
    EmploymentConverter.class, 
    SubscriptionConverter.class
})
public abstract class UserDatabase extends RoomDatabase {

    public abstract UserDao userDao();

    private static final int NUMBER_OF_THREADS = 4;
    public static final ExecutorService databaseWriteExecutor =
            Executors.newFixedThreadPool(NUMBER_OF_THREADS);
}

Do  klasy `AppModule` dodajmy metodę dostarczającą bazę danych

In [ ]:
    @Provides
    @Singleton
    UserDatabase provideUserDatabase(Application app){
        return Room
            .databaseBuilder(
                app, 
                UserDatabase.class, 
                "user_database_java4")
            .build();
    }

Podczasc pobierania danych mogą wystąpić trzy statusy - sukces, error, loading - dla pomyślnego zakończenia pobierania danych, błędu i czasu łączenia i pobierania.

In [ ]:
public enum Status {
    SUCCESS,
    ERROR,
    LOADING
}

Dodajmy klasę `Resources`, która pozwoli nam wyemitować odpowiedni stan.

In [ ]:
public class Resource<T> {}

Klasa posiada trzy pola przechowujące status, dane oraz komunikat (otrzymywany w przypadku błędu)

In [ ]:
@NonNull
public final Status status;
@Nullable
public final T data;
@Nullable public final String message;

Klasa posiada konstruktor prywatny

In [ ]:
private Resource(@NonNull Status status, @Nullable T data, @Nullable String message) {
    this.status = status;
    this.data = data;
    this.message = message;
}

Ostatnią częścią, są trzy metody tworzące odpowiednia skonfigurowany obiekt `Resource`

In [ ]:
public static <T> Resource<T> success(@NonNull T data) {
    return new Resource<>(SUCCESS, data, null);
}

public static <T> Resource<T> error(String msg, @Nullable T data) {
    return new Resource<>(ERROR, data, msg);
}

public static <T> Resource<T> loading(@Nullable T data) {
    return new Resource<>(LOADING, data, null);
}

Zdefiniujmy klasę abstrakcyjną `NetworkBoundResource`, bedzie ona odpowiedzialna za logikę dostępu do danych - kiedy i w jakich warunkach dostajemy dane z serwera lub bazy lokalnej.

In [ ]:
public abstract class NetworkBoundResource<ResultType, RequestType> {

Klasa posiada dwa generyczne argumenty `ResultType` i `RequestType` -  dwa, ponieważ typ danych otrzymanych z api może się różnić od typu otrzymanego z lokalnej bazy - w tej aplikacji typ jest taki sam.

Dodajmy jedno pole typ `MediatorLiveData` - klasa ta umożliwia obserwację kilku obiektów `LiveData` i reagować na zmiany.

```java
   LiveData  liveData1 = ...;
   LiveData  liveData2 = ...;
  
   MediatorLiveData  liveDataMerger = new MediatorLiveData<>();
   liveDataMerger.addSource(liveData1, value -> liveDataMerger.setValue(value));
   liveDataMerger.addSource(liveData2, value -> liveDataMerger.setValue(value));
```

In [ ]:
private final MediatorLiveData<Resource<ResultType>> result = new MediatorLiveData<>();

Ponieważ tworzymy klasę abstrakcyjną, w repozytorium utworzymy **klasę anonimową** o typie `NetworkBoundResource` i tam dostarczymy implementacje niektórych z poniższych metod.
Zdefiniujmy metodę abstrakcyjną zwracającą `LiveData` z bazy lokalnej.

In [ ]:
@NonNull
protected abstract LiveData<ResultType> loadFromDb();

Zdefiniujmy również metodę tworzącą obiekt `Call`

In [ ]:
@NonNull
protected abstract Call<RequestType> createCall();

Kolejna metoda służy do zapisania obiektu `Call` w lokalnej bazie

In [ ]:
protected abstract void saveCallResult(@NonNull RequestType item);

Metoda `shouldFetch` dostarcza informacji o tym czy chcemy aktualizować dane z serwera - domyślnie zwraca `true`

In [ ]:
protected boolean shouldFetch(@Nullable ResultType data) {
    return true;
}

Zdefiniujmy również metodę odpowiedzielną za niepowodzenie pobrania danych z serwera.

In [ ]:
protected void onFetchFailed() {}

Dodajmy metodę zwracającą `LiveData` na podstawie obiektu `MediatorLiveData`

In [ ]:
public final LiveData<Resource<ResultType>> getAsLiveData() {
    return result;
}

Następnie, dodajmy metodę `saveResultAndReInit` zapisującą wynik do bazy lokalnej oraz dodającą nową wersję `LiveData` do `MediatorLiveData`

In [ ]:
private void saveResultAndReInit(RequestType response) {
    UserDatabase.databaseWriteExecutor.execute(() -> saveCallResult(response));
    result.addSource(loadFromDb(), newData -> result.setValue(Resource.success(newData)));
}

W pierwszym kroku wykonujemy asynchronicznie metodę `saveCallResult` - zapisujemy w bazie wynik żądania z serwera. Następnie dodajemy nowe źródło do `MediatorLiveData` - wywołujemy metodę `loadFromDb` zwracającą `LiveData`, następnie ten obiekt (odnosimy się do niego jako `newData`) przekazujemy do `MediatorLiveData` wykonując metodę `setValue` - jest on *opakowany* w `Resource`.

Ostatnią metodą jest `fetchFromNetwork` przyjmująca jeden argument typu `LiveData`.

In [ ]:
private void fetchFromNetwork(final LiveData<ResultType> dbSource) {

Wpierw dodajemy to co otrzymujemy z bazy jako źródło do `MediatorLiveData`

In [ ]:
result.addSource(dbSource, newData -> result.setValue(Resource.loading(newData)));

Następnie wywołujemy metodę `createCall` i wywołujemy metodę `enqueue`

In [ ]:
createCall().enqueue(new Callback<RequestType>() {

W metodzie `onResponse` usuwamy stare źródło danych z `MediatorLiveData` i wywołujemy metodę `saveResultAndReInit` - czyli zamieniamy źródło.

In [ ]:
@Override
public void onResponse(
    @NonNull Call<RequestType> call, 
    @NonNull Response<RequestType> response) {
    result.removeSource(dbSource);
    saveResultAndReInit(response.body());
}

W metodzie `onFailure` wywołujemy metodę `onFetchFail`, usuwamy źródło i dodajemy je ponownie lecz *opakowane* w `Resource.error` - w przypadku niepowodzenia dalej posługujemy się wersją bazy dostępną lokalnie.

In [ ]:
@Override
public void onFailure(
    @NonNull Call<RequestType> call, 
    @NonNull Throwable t) {
    onFetchFailed();
    result.removeSource(dbSource);
    result.addSource(dbSource, 
                     newData -> result.setValue(Resource.error(t.getMessage(), newData)));
}

Ostatnim elementem jest konstruktor

In [ ]:
protected NetworkBoundResource() {
    result.setValue(Resource.loading(null));
    LiveData<ResultType> dbSource = loadFromDb();
    result.addSource(dbSource, data -> {
        result.removeSource(dbSource);
        if (shouldFetch(data)) {
            fetchFromNetwork(dbSource);
        } else {
            result.addSource(dbSource, 
                             newData -> result.setValue(Resource.success(newData)));
        }
    });
}

Gdy wywołujemy konstruktor w pierwszej kolejności emitujemy stan `Resource.loading`, następnie ładujemy dane z lokalnej bazy i dodajemy źródło. Tym razem sprawdzamy czy powinniśmy wykonać żądanie z serwera. Jeżeli tak, wykonujemyn metodę `fetchFromNetwork`m jeżeli nie - ponownie dodajemy źródło i emitujemy stan `Resource.success`

Dodajmy repozytorium, wykorzystamy wstrzyknięcie przez konstruktor `RandomApi` oraz `UserDatabase`

In [ ]:
public class UserRepository {

    private final UserDao dao;
    private final RandomApi api;

    @Inject
    public UserRepository(UserDao dao, RandomApi api) {
        this.dao = dao;
        this.api = api;
    }
}

W repozytorium zdefiniujmey jedną funkcję `getUsers`, w której wywołamy poprzednio zaimplementowaną `networkBoundResource`

In [ ]:
public LiveData<Resource<List<User>>> getUsers() {

Tworzymy **klasę anonimową**

In [ ]:
return new NetworkBoundResource<List<User>, List<User>>() {

Dodajmy implementację metod zdefiniowanych w klasie `NetworkBoundResource`

In [ ]:
@Override
protected void saveCallResult(@NonNull List<User> item) {
    dao.clear();
    dao.insert(item);
}

@NonNull
@Override
protected LiveData<List<User>> loadFromDb() {return dao.getUsers();}

@NonNull
@Override
protected Call<List<User>> createCall() {
    return api.users();
}

Ponieważ wywołujemy dwie funkcje: `clear` - czyszcząca bazę, oraz `insert` - dodająca całą listę.

Pełny kod repozytorium:

In [ ]:
public class UserRepository {

    private final UserDao dao;
    private final RandomApi api;

    @Inject
    public UserRepository(UserDao dao, RandomApi api) {
        this.dao = dao;
        this.api = api;
    }

    public LiveData<Resource<List<User>>> getUsers() {
        return new NetworkBoundResource<List<User>, List<User>>() {

            @Override
            protected void saveCallResult(@NonNull List<User> item) {
                dao.clear();
                dao.insert(item);
            }

            @NonNull
            @Override
            protected LiveData<List<User>> loadFromDb() {return dao.getUsers();}

            @NonNull
            @Override
            protected Call<List<User>> createCall() {
                return api.users();
            }
        }.getAsLiveData();
    }
}

Na końcu wywołujemy `getAsLiceData` aby otrzymać obiekt typu `LiveData` a nie `MediatorLiveData`

Dodajmy `ViewModel`

In [ ]:
@HiltViewModel
public class UserViewModel extends ViewModel {
    private LiveData<Resource<List<User>>> users;
    private final UserRepository repository;

    @Inject
    public UserViewModel(UserRepository repository) {
        this.repository = repository;

    }

    public LiveData<Resource<List<User>>> getUsers() {
        return repository.getUsers();
    }
}

W aktywności fgłównej posiadamy `RecyclerView`, więc dodajmyn `Comparator`, `ViewHolder` oraz `Adapter`

In [ ]:
public class UserComparator extends DiffUtil.ItemCallback<User> {
    @Override
    public boolean areItemsTheSame(@NonNull User oldItem, @NonNull User newItem) {
        return newItem.getUid().equals(oldItem.getUid());
    }

    @Override
    public boolean areContentsTheSame(@NonNull User oldItem, @NonNull User newItem) {
        return newItem.getId() == oldItem.getId();
    }
}

public class UserViewHolder extends RecyclerView.ViewHolder {

    private final RvItemBinding binding;

    public UserViewHolder(RvItemBinding binding) {
        super(binding.getRoot());
        this.binding = binding;
    }

    public void bind(User item){
        binding.firstName.setText(item.getFirst_name());
        binding.lastName.setText(item.getLast_name());
        binding.city.setText(item.getAddress().getCity());
        binding.country.setText(item.getAddress().getCountry());
        binding.dateOfBirth.setText(item.getDate_of_birth());
        binding.creditCardNumber.setText(item.getCredit_card().getCc_number());
        binding.email.setText(item.getEmail());
        binding.employment.setText(item.getEmployment().getTitle());
        binding.gender.setText(item.getGender());
        binding.password.setText(item.getPassword());
        binding.paymentMethod.setText(item.getSubscription().getPayment_method());
        binding.phone.setText(item.getPhone_number());
        binding.plan.setText(item.getSubscription().getPlan());
        binding.term.setText(item.getSubscription().getTerm());
        binding.paymentMethod.setText(item.getSubscription().getPayment_method());
        binding.state.setText(item.getAddress().getState());
        binding.zipCode.setText(item.getAddress().getZip_code());
        binding.status.setText(item.getSubscription().getStatus());
        binding.streetAddress.setText(item.getAddress().getStreet_address());
        binding.streetName.setText(item.getAddress().getStreet_name());
        binding.username.setText(item.getUsername());
        Glide.with(binding.getRoot()).load(item.getAvatar()).into(binding.image);
    }
}

    
public class UserAdapter extends ListAdapter<User, UserViewHolder> {
    public UserAdapter(UserComparator comparator) {
        super(comparator);
    }

    @NonNull
    @Override
    public UserViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new UserViewHolder(RvItemBinding.inflate(
                LayoutInflater.from(parent.getContext()), parent, false
        ));
    }

    @Override
    public void onBindViewHolder(@NonNull UserViewHolder holder, int position) {
        User item = getItem(position);
        holder.bind(item);
    }
}


Możemy przetestować aplikację. Jak widzimy przy pierwszym uruchomieniu ładowane są dane, przy drugim wyświetlane są dane *offline cached* z bazy lokalnej, asynchronicznie wykonywana jest aktualizacja z serwera.

<img src="https://media1.giphy.com/media/y0RLxRN7SPJ7mI7Hw5/giphy.gif?cid=790b7611ea236e1ec92b9220e3629a80e851a7759882cbee&rid=giphy.gif&ct=g" width="150" />